In [1]:
import itertools
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import NonlinearConstraint
from scipy.optimize import SR1
from sinkhorn_knopp import sinkhorn_knopp as skp
from scipy.optimize import show_options
from scipy.special import comb

In [2]:
#@Author: Rajit Datta
# Takes a list of n values and computes 
# the kth elementary symmetric polynomial of the list L (assuming n >=k) using newton identities
def comp_snk(L,k):
    n=len(L)
    snklist=[1]
    powersums=[n]
    for i in range(1,k+1):
        powersums.append(sum([x**i for x in L]))

    for t in range(1,k+1):
        s=0
        for i in range(1,t+1):
            s=s+((-1)**(i-1))*snklist[t-i]*powersums[i]    
        snklist.append((s/float(t)))
    return snklist[-1]

# takes a bin(i) type of input for 
# i < 2^n and returns an integer list of length n containing 0,1 at the bit positions
def conv(s,n): 
    sts=s[2:]
    st="0"*(n - len(sts))+sts
    #print(st)
    lis=map(int , map(lambda x: x+"" , st))
    return(list(lis))

def factorial(i): #computes i!
    f=1
    for j in range(1,i+1):
        f=f*j
    return f

#Computes rectangular permanent of k x n matrix where k <= n. Takes a numpy matrix as input
def permanent(B): 
    k=np.size(B,0)
    n=np.size(B,1)
    if k > n: #Transpose the matrix if needed
        B=B.T

    k=np.size(B,0)
    n=np.size(B,1)
    p=0 # p for permanent
    ao=np.matrix([1]*k) #all 1 vector

    for i in range(0,2**k): #Implements Ryser formula and apolarity
        st=conv(bin(i),k)
        x=np.matrix(st)
        vec=(x*B).tolist()[0]
        p=p + ((-1.0)**((ao*x.transpose())[0,0]))*comp_snk(vec,k)
#         if np.mod(i,(2**k)/100)==0:
#             print(i/100)

    return (((-1)**k)*p)

In [3]:
import warnings

import numpy as np


class SinkhornKnoppR:

    def __init__(self, max_iter=1000, epsilon=1e-3):
        assert isinstance(max_iter, int) or isinstance(max_iter, float),\
            "max_iter is not of type int or float: %r" % max_iter
        assert max_iter > 0,\
            "max_iter must be greater than 0: %r" % max_iter
        self._max_iter = int(max_iter)

        assert isinstance(epsilon, int) or isinstance(epsilon, float),\
            "epsilon is not of type float or int: %r" % epsilon
        assert epsilon > 0 and epsilon < 1,\
            "epsilon must be between 0 and 1 exclusive: %r" % epsilon
        self._epsilon = epsilon

        self._stopping_condition = None
        self._iterations = 0
        self._D1 = np.ones(1)
        self._D2 = np.ones(1)

    def fit(self, P):
        """Fit the diagonal matrices in Sinkhorn Knopp's algorithm
        Parameters
        ----------
        P : 2d array-like
        Must be a square non-negative 2d array-like object, that
        is convertible to a numpy array. The matrix must not be
        equal to 0 and it must have total support for the algorithm
        to converge.
        Returns
        -------
        A double stochastic matrix.
        """
        P = np.asarray(P)
        assert np.all(P >= 0)
        assert P.ndim == 2
#         assert P.shape[0] == P.shape[1]

        N = P.shape[0]
        max_thresh = 1 + self._epsilon
        min_thresh = 1 - self._epsilon

        # Initialize r and c, the diagonals of D1 and D2
        # and warn if the matrix does not have support.
        r = np.ones((N, 1))
        pdotr = P.T.dot(r)
#         print('pdotr: ', pdotr)
        total_support_warning_str = (
            "Matrix P must have total support. "
            "See documentation"
        )
        if not np.all(pdotr != 0):
            warnings.warn(total_support_warning_str, UserWarning)
        
        c = 1 / pdotr
#         print('c: ',c)
        pdotc = P.dot(c)
#         print('pdotc: ',pdotc)
        if not np.all(pdotc != 0):
            warnings.warn(total_support_warning_str, UserWarning)

        
        r = 1 / pdotc
#         print('r: ',r.shape)
        del pdotr, pdotc

        P_eps = np.copy(P)
        while (np.any(np.sum(P_eps, axis=1) < min_thresh) \
                or np.any(np.sum(P_eps, axis=1) > max_thresh) \
                or np.any(np.sum(P_eps, axis=0) < min_thresh) \
                or np.any(np.sum(P_eps, axis=0) > max_thresh))\
                and np.any(~(r<1e-300)):
            
            
            c = 1 / P.T.dot(r)
            r = 1 / P.dot(c)
            
#             print('c: ',c, '\n', 'r: ',r)
            
            self._D1 = np.diag(np.squeeze(r))
            self._D2 = np.diag(np.squeeze(c))
            P_eps = self._D1.dot(P).dot(self._D2)
#             print('P_eps: ',P_eps)
            self._iterations += 1

            if self._iterations >= self._max_iter:
                self._stopping_condition = "max_iter"
                break

        if not self._stopping_condition:
            self._stopping_condition = "epsilon"

        self._D1 = np.diag(np.squeeze(r))
        self._D2 = np.diag(np.squeeze(c))
        P_eps = self._D1.dot(P).dot(self._D2)

        return P_eps

In [4]:
def f_bp(P):
    
    def f_bp_p(beta):
        k=P.shape[0]
        n=P.shape[1]
        z=0
#         print(beta.sum())
        beta=beta.reshape(k,n)
#         print(beta)
        for i in range(k):
            for j in range(n):
                
#                 if beta[i,j]<0:
#                     print(beta[i,j])
                if beta[i,j]<1 and beta[i,j]>0 and P[i,j]>0: 
                    z+=beta[i,j]*np.log(beta[i,j]/P[i,j])-(1-beta[i,j])*np.log(1-beta[i,j])
#                     print(z)
                
        return z
    return f_bp_p

In [5]:
def constraint_matrix(n,k):
    m=np.zeros([n+k,n*k])
    for i in range(n):
        m[i,i*k:i*k+k]=1
    for i in range(k):
        for j in range(i,n*k,k):
            m[i+n,j]=1
    
    return m

In [6]:
np.random.seed(69);
P=np.random.randint(1000,size=(8,20));#P=np.kron(np.eye(2,2),np.ones((2,2)))
temp=np.random.rand(P.shape[0],P.shape[1]);
for i in range(P.shape[0]):
    for j in range(P.shape[1]):
        if P[i,j]==0:
            temp[i,j]=0

In [8]:
sk = SinkhornKnoppR()
temp_ds=sk.fit(temp);
f=f_bp(P);
m=constraint_matrix(P.shape[0],P.shape[1]);
b1=np.append([np.ones(P.shape[0])], [np.zeros(P.shape[1])]);
b2=np.append([np.ones(P.shape[0])], [np.ones(P.shape[1])]);
linear_constraint=LinearConstraint(m,b1,b2)
bounds = Bounds(0,1);

In [11]:
res = minimize(f, temp_ds.flatten(), method='trust-constr',
...                constraints=[linear_constraint],
...                options={'verbose': 1, 'maxiter':1000, 'xtol':1e-17, 'gtol':1e-17}, bounds=bounds)

/home/ativ/cmi/lib/python3.5/site-packages/scipy/optimize/_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)


`xtol` termination condition is satisfied.
Number of iterations: 251, function evaluations: 44114, CG iterations: 4424, optimality: 1.47e-08, constraint violation: 2.22e-16, execution time: 9.1e+01 s.


In [12]:
t=(res.x).reshape(P.shape[0],P.shape[1])
n=P.shape[0]
m=P.shape[1]
lb=np.exp(-f(t));
print('lb: ', lb)#format(lb,".2E")
ub=lb*(np.sqrt(2)**P.shape[0]);print('ub: ',ub)
per=permanent(P);print('per:',per)
if lb<=per<=ub:
    print(True)
else:
    print(False)

lb:  3.3314667159371913e+28
ub:  5.330346745499508e+29
per: 1.4971437402087266e+31
False


In [ ]:

np.random.seed(69);
P=np.random.randint(1000,size=(3,5));#P=np.kron(np.eye(2,2),np.ones((2,2)))
temp=np.random.rand(P.shape[0],P.shape[1]);
for i in range(P.shape[0]):
    for j in range(P.shape[1]):
        if P[i,j]==0:
            temp[i,j]=0

sk = SinkhornKnoppR()
temp_ds=sk.fit(temp);
f=f_bp(P);
m=constraint_matrix(P.shape[0],P.shape[1]);
b1=np.append([np.ones(P.shape[0])], [np.zeros(P.shape[1])]);
b2=np.append([np.ones(P.shape[0])], [np.ones(P.shape[1])]);
linear_constraint=LinearConstraint(m,b1,b2)
bounds = Bounds(0,1);

res = minimize(f, temp_ds.flatten(), method='trust-constr',
...                constraints=[linear_constraint],
...                options={'verbose': 1, 'maxiter':1000, 'xtol':1e-8}, bounds=bounds)

t=(res.x).reshape(P.shape[0],P.shape[1])
n=P.shape[0]
m=P.shape[1]
lb=np.exp(-f(t))/(n**(m-n));
print('lb: ', lb)#format(lb,".2E")
ub=lb*(np.sqrt(2)**P.shape[0]);print('ub: ',ub)
per=permanent(P);print('per:',per)
if lb<=per<=ub:
    print(True)
else:
    print(False)

In [22]:
t=(res.x).reshape(P.shape[0],P.shape[1])

lb=np.exp(-f(t));
print('lb: ', lb)#format(lb,".2E")
ub=np.exp(-f(t))*(np.sqrt(2)**P.shape[0]);print('ub: ',ub)
per=permanent(P);print('per:',per)
if lb<=per<=ub:
    print(True)
else:
    print(False)

lb:  207882295635.55002
ub:  1175959847460.1929
per: 5954357867753.0
False


In [61]:
P

array([[54, 75, 73, 90, 55],
       [20, 49, 22,  9, 56],
       [97, 38, 96, 88, 12]])

In [63]:
P

array([[ 54, 203, 969, 619, 602],
       [439, 404, 561, 278,   9],
       [184, 225, 806, 352,  88]])

In [10]:
x=5
np.random.seed(0)
rnd=np.random.randint(1000)

for nsz in range(2,10):
    data=np.zeros((x,4))
    np.random.seed(rnd+nsz)
    sd=np.random.randint(1000)
    for s in range(x):
        np.random.seed(sd+s);
        P=np.random.randint(100,size=(nsz,20));
        temp=np.random.rand(P.shape[0],P.shape[1]);
        for i in range(P.shape[0]):
            for j in range(P.shape[1]):
                if P[i,j]==0:
                    temp[i,j]=0

        sk = SinkhornKnoppR()
        temp_ds=sk.fit(temp);
        f=f_bp(P);
        m=constraint_matrix(P.shape[0],P.shape[1]);
        b=np.append([np.ones(P.shape[0])], [(P.shape[0]/P.shape[1])*np.ones(P.shape[1])]);
        linear_constraint=LinearConstraint(m,b,b)
        bounds = Bounds(0,1);[(P.shape[0]/P.shape[1])*np.ones(P.shape[1])]

        res = minimize(f, temp_ds.flatten(), method='trust-constr',constraints=[linear_constraint],
                       options={'verbose': 0, 'maxiter':1000, 'xtol':1e-8}, bounds=bounds)

        t=(res.x).reshape(P.shape[0],P.shape[1])
        print(nsz,s)
        lb=np.exp(-f(t));print('lb: ', lb)
        ub=np.exp(-f(t))*(2**P.shape[0]);print('ub: ',ub)
        per=permanent(P);print('per:',per)
        print('ratio: ',np.log(per/lb))
        if lb<=per<=ub:
            print(True)
        else:
            print(False)
        data[s][0]=lb
        data[s][1]=per
        data[s][2]=np.log(per/lb)
        
#     np.save('recdata_per_'+str(nsz)+'x8',data)

2 0
lb:  133378.25381397162
ub:  533513.0152558865
per: 1144560.0
ratio:  2.149586457352695
False
2 1
lb:  86362.83143493543
ub:  345451.32573974173
per: 861280.0
ratio:  2.2998622632869283
False


KeyboardInterrupt: 

In [3]:
print('n x 20, log_2(permanent/bethe)')
# print('n x 20, ln(permanent/bethe)')
for i in range(2,21):
    data=np.load('recdata_per_'+str(i)+'x8.npy')
#     print(i)
#     print(data)
#     print(i, np.log2(np.exp(data[:,2])))
    print(i,data[:,2])

n x 20, log_2(permanent/bethe)
2 [2.12200199 2.34085168 2.22003451 2.02001501 2.06832992]
3 [3.0782564  2.98171849 2.88705727 2.97635083 2.88269356]
4 [3.81903994 3.69227422 3.91749424 3.77767584 3.66804381]
5 [4.46997285 4.45050437 4.48742504 4.53379769 4.51953024]
6 [5.12949594 5.11042022 5.32027569 5.1294577  5.25690383]
7 [5.71451547 5.70962237 5.73029736 5.79168611 5.77434521]
8 [6.35160565 6.28168256 6.32915938 6.28123565 6.23844069]
9 [6.79095566 6.720445   6.78162385 6.72631602 6.77099628]
10 [7.05948458 7.1292291 ]
11 [7.36743333 7.3685987 ]
12 [7.55009927 7.56997148]
13 [7.72146221 7.65810379]
14 [7.54750278 8.22572586]
15 [7.2665323  7.29558576]
16 [6.93595292 6.91920016]
17 [6.32420807 6.30336669]
18 [5.63402716 7.07970354]
19 [3.95472021 3.93274911]
20 [6.02310363 2.37241397]


In [36]:
print('n x 10, log_2(permanent/bethe)')
# print('n x 20, ln(permanent/bethe)')
for i in range(2,10):
    data=np.load('recdata_per_'+str(i)+'x10.npy')
#     print(i)
#     print(data)
    print(i, np.log2(np.exp(data[:,2])))
#     print(i,data[:,2])

n x 10, log_2(permanent/bethe)
2 [3.55490336 3.08186945]
3 [3.66580443 3.80432655]
4 [4.80683956 4.65585296]
5 [5.25112295 5.19444356]
6 [5.56323546 5.56078804]
7 [5.5701377  5.55740887]
8 [5.1833171  5.17975432]
9 [4.22795663 4.31617138]
